In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,ConfusionMatrixDisplay
from sklearn.decomposition import PCA 
import seaborn as sb
from sklearn.preprocessing import StandardScaler,LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense,LSTM,SimpleRNN
import keras_tuner

df = pd.read_csv("emotions.csv")

x = df.loc[:,"fft_0_b":"label"]
y = df['label']
x.drop(columns='label',inplace=True,axis=1)
n_optimal = 45
model = PCA(n_components=45)
transformed_data =PCA.fit_transform(model,X=x)
scale = StandardScaler() 
x = scale.fit_transform(transformed_data)

encode = LabelEncoder()
y_ = encode.fit_transform(y)
x_train,x_test,y_train,y_test = train_test_split(x,y_,train_size=0.8,random_state=42)


In [4]:
activation  = ['relu',
               'tanh']
Loss =        ['binary_crossentropy',
               'sparse_categorical_crossentropy',
               'categorical_crossentropy',
               ]
Optimizer =   ['adam',
               'sgd',
               'rmsprop',
               'adadelta']

def build_model(hp):
  model = Sequential()

  model.add(LSTM(units = 45,
                 activation=hp.Choice('activation_1',activation),
                 return_sequences=True,
                 input_shape=(45, 1)))
  model.add(LSTM(units = 30,
                      activation=hp.Choice('activation_2',activation))) 
  model.add(Dense(3, 
                      activation='softmax')) 
  optimize = hp.Choice('optimizer',
                       values = Optimizer)
  loss     = hp.Choice('loss',
                       values = Loss)
  
  model.compile(optimizer=optimize,
                loss= loss,
                metrics = ['accuracy']
                )
  return model

tune = keras_tuner.GridSearch(hypermodel=build_model,
                              objective='val_accuracy',
                              overwrite=True,
                              directory='LSTM_HyperParameter_Data_With_PCA',
                              project_name='OptimizerAndLoss')
tune.search(x_train,y_train,epochs=15,validation_data = (x_test,y_test))

Trial 48 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.868852436542511
Total elapsed time: 01h 07m 15s
INFO:tensorflow:Oracle triggered exit


In [9]:
hyp_param = tune.get_best_hyperparameters(num_trials=1)[0].values
print(hyp_param)

{'unit1': 65, 'unit2': 65}


In [6]:
def build_model(hp):
  model = Sequential()
  unit1 = hp.Int('unit1',min_value = 45, max_value = 65, step = 10)
  unit2 = hp.Int('unit2',min_value = 45, max_value = 90, step = 10)
  model.add(LSTM(units = unit1,
                 activation=hyp_param['activation_1'],
                 return_sequences=True,
                 input_shape=(45, 1)))
  model.add(LSTM(units = unit2,
                      activation=hyp_param['activation_2'])) 
  model.add(Dense(3, 
                      activation='softmax')) 
  optimize = hyp_param['optimizer']
  loss     = hyp_param['loss']
  
  model.compile(optimizer=optimize,
                loss= loss,
                metrics = ['accuracy']
                )
  return model

tune = keras_tuner.GridSearch(hypermodel=build_model,
                              objective='val_accuracy',
                              overwrite=True,
                              directory='LSTM_HyperParameter_Data_With_PCA',
                              project_name='Units')
tune.search(x_train,y_train,epochs=10,validation_data = (x_test,y_test))

Trial 15 Complete [00h 02m 00s]
val_accuracy: 0.8548009395599365

Best val_accuracy So Far: 0.8735362887382507
Total elapsed time: 00h 12m 00s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tune.get_best_models(num_models=1)[0]
history = model.fit(x_train,y_train,epochs=15,batch_size=10)
model.save("LSTM_with_pca.h5")
